In [5]:
# Importacao das bibliotecas

import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [6]:
# Lendo o arquivo

df = pd.read_csv('/content/dataframe.csv')
df.sample(5)

,data_medicao,nome_condominio,quantidade_placas,potencia_placa_kW,consumo_kWh,quantidade_habitantes,energia_solar_gerada_kWh,energia_recomendada_kWh
138443,2031-07-31,Condomínio_44,45,0.35,1516.62,140,48.94,5055.39
40503,2026-03-21,Condomínio_4,56,0.35,4410.20,389,86.17,14700.68
116245,2030-05-13,Condomínio_46,34,0.30,3393.79,255,38.87,11312.62
122705,2030-09-20,Condomínio_6,38,0.35,1400.78,113,52.48,4669.28
125297,2030-11-10,Condomínio_48,41,0.30,4666.58,435,53.86,15555.27


In [7]:
# Vendo informações sobre o dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   data_medicao              150000 non-null  object 
 1   nome_condominio           150000 non-null  object 
 2   quantidade_placas         150000 non-null  int64  
 3   potencia_placa_kW         150000 non-null  float64
 4   consumo_kWh               150000 non-null  float64
 5   quantidade_habitantes     150000 non-null  int64  
 6   energia_solar_gerada_kWh  150000 non-null  float64
 7   energia_recomendada_kWh   150000 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 9.2+ MB


## Detalhes do dataset

Podemos verificar que o dataset é bem completo e não possui colunas nulas, isso vai ajudar a treinar nosso modelo.

### Informações sobre as colunas

 - data_medicao  
   A data que foi tirado aquela medição.

 - nome_condominio  
   O nome do condomínio.

 - quantidade_placas  
   A quantidade de placas que o condomínio tinha no dia que foi tirado aquela medição.

 - potencia_placa_kW  
   A potência daquela placa no dia daquela medição.

 - consumo_kWh:  
   O consumo de energia do condomínio naquele dia.

 - quantidade_habitantes:  
   A quantidade de habitantes que tinha no condomínio naquele dia.

 - energia_solar_gerada_kWh:  
   A energia que estava sendo gerada naquele dia.

 - energia_recomendada_kWh:  
   A quantidade de energia recomendada



In [8]:
df.head()

,data_medicao,nome_condominio,quantidade_placas,potencia_placa_kW,consumo_kWh,quantidade_habitantes,energia_solar_gerada_kWh,energia_recomendada_kWh
0,2024-01-01,Condomínio_1,35,0.35,2439.63,219,60.18,8132.09
1,2024-01-01,Condomínio_2,46,0.35,5511.82,390,77.76,18372.74
2,2024-01-01,Condomínio_3,50,0.35,3706.46,336,77.45,12354.85
3,2024-01-01,Condomínio_4,55,0.30,5014.34,389,80.96,16714.45
4,2024-01-01,Condomínio_5,45,0.30,4636.33,325,65.70,15454.43
